In [1]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode

In [2]:
import numpy as np
from shapely.geometry import Point, Polygon

In [3]:
import geopandas as gpd

In [4]:
ruta_archivo = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/estaciones INM/inm_gdp.csv'

# Cargar el archivo .xls en un DataFrame
df = pd.read_csv(ruta_archivo)

In [6]:
grid = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/grid.csv')


In [10]:
grid

,ID_Poligono,geometry
0,PG1,"POLYGON ((-120.00000 10.00000, -119.00000 10.0..."
1,PG2,"POLYGON ((-120.00000 11.00000, -119.00000 11.0..."
2,PG3,"POLYGON ((-120.00000 12.00000, -119.00000 12.0..."
3,PG4,"POLYGON ((-120.00000 13.00000, -119.00000 13.0..."
4,PG5,"POLYGON ((-120.00000 14.00000, -119.00000 14.0..."
...,...,...
1021,PG1022,"POLYGON ((-83.00000 32.00000, -82.00000 32.000..."
1022,PG1023,"POLYGON ((-83.00000 33.00000, -82.00000 33.000..."
1023,PG1024,"POLYGON ((-83.00000 34.00000, -82.00000 34.000..."
1024,PG1025,"POLYGON ((-83.00000 35.00000, -82.00000 35.000..."


### Procesamiento

1) Asegúrate de que 'grid' es un DataFrame y tiene una columna 'geometry' que podría estar en formato de cadena

In [9]:
from shapely.wkt import loads
if 'geometry' in grid.columns:
    # Convertir cadenas de geometría a objetos geométricos usando `loads` de Shapely
    grid['geometry'] = grid['geometry'].apply(lambda x: loads(x) if isinstance(x, str) else x)

    # Ahora que todos los valores en 'geometry' son objetos geométricos, crea el GeoDataFrame
    grid = gpd.GeoDataFrame(grid, geometry='geometry')
else:
    print("La columna 'geometry' no existe en 'grid'.")

2) hacemos un join con los puntos xy de cada ubicacion y le asignamos su ID_poligono en el grid

In [11]:
# Convertir df1 en GeoDataFrame
gdf1 = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitud, df.latitud))

# Aquí estoy asumiendo que tus datos están en WGS84 (EPSG:4326)
gdf1.set_crs(epsg=4326, inplace=True)
grid.set_crs(epsg=4326, inplace=True)

# Realizar la operación sjoin
gdf1_sjoined = gpd.sjoin(gdf1, grid, how="left", op="within")

# Asignar los 'ID_Poligono' a df1 desde el resultado del sjoin
df["ID_Poligono"] = gdf1_sjoined["ID_Poligono"]

# Verificar los resultados
print(df.head())

                                                name tipo  \
0                       Acayucan Estacion Migratoria  INM   
1                   Agua Prieta Estacion Provisional  INM   
2                 Aguascalientes Estacion Migratoria  INM   
3                      Campeche Estacion Provisional  INM   
4  Centro de Atencion Integral al Trafico Fronter...  INM   

                      sub_tipo    latitud    longitud ID_Poligono  
0      Estacion Migratoria INM  18.001851  -94.921076       PG685  
1      Estacion Migratoria INM  31.332947 -109.560374       PG293  
2      Estacion Migratoria INM  21.929551 -102.320223       PG472  
3      Estacion Migratoria INM  19.845660  -90.539741       PG794  
4  Centro de Atencion Integral  16.149934  -92.072111       PG737  


/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [13]:
# Realizar el merge (unión) entre datos_csv2 y grid usando 'ID_Poligono' como clave
df = df.merge(grid[['ID_Poligono', 'geometry']], on='ID_Poligono', how='left')

guardamos

In [15]:
ruta_nueva2 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/estaciones INM/inm_gdp.csv'

# Guardar el DataFrame en un archivo Excel en la ruta especificada
df.to_csv(ruta_nueva2, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_nueva2}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/estaciones INM/inm_gdp.csv


In [66]:
ruta_archivo1 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/estaciones INM/inm_gdp.csv'

# Cargar el archivo .xls en un DataFrame
df1 = pd.read_csv(ruta_archivo1)

In [67]:
# Eliminar la columna 'geometry'
df1 = df1.drop(columns=['geometry'])

# Verificar el resultado mostrando las primeras filas del DataFrame
print(df1.head())


                                                name tipo  \
0                       Acayucan Estacion Migratoria  INM   
1                   Agua Prieta Estacion Provisional  INM   
2                 Aguascalientes Estacion Migratoria  INM   
3                      Campeche Estacion Provisional  INM   
4  Centro de Atencion Integral al Trafico Fronter...  INM   

                      sub_tipo    latitud    longitud ID_Poligono  
0      Estacion Migratoria INM  18.001851  -94.921076       PG685  
1      Estacion Migratoria INM  31.332947 -109.560374       PG293  
2      Estacion Migratoria INM  21.929551 -102.320223       PG472  
3      Estacion Migratoria INM  19.845660  -90.539741       PG794  
4  Centro de Atencion Integral  16.149934  -92.072111       PG737  


In [76]:
df1.head(60)

,name,tipo,sub_tipo,latitud,longitud,ID_Poligono,ciudad,estado
0,Acayucan Estacion Migratoria,INM,Estacion Migratoria INM,18.001851,-94.921076,PG685,Acayucan,Veracruz
1,Agua Prieta Estacion Provisional,INM,Estacion Migratoria INM,31.332947,-109.560374,PG293,Agua Prieta,Sonora
2,Aguascalientes Estacion Migratoria,INM,Estacion Migratoria INM,21.929551,-102.320223,PG472,Aguascalientes,Aguascalientes
3,Campeche Estacion Provisional,INM,Estacion Migratoria INM,19.845660,-90.539741,PG794,Campeche,Campeche
4,Centro de Atencion Integral al Trafico Fronter...,INM,Centro de Atencion Integral,16.149934,-92.072111,PG737,La Trinitaria,Chiapas
5,Centro de Atencion Integral al Transito Fronte...,INM,Centro de Atencion Integral,15.162749,-92.532987,PG736,Salto de Agua,Chiapas
6,Chetumal Estacion Migratoria,INM,Estacion Migratoria INM,18.526295,-88.283332,PG847,Chetumal,Quintana Roo
7,Chihuahua Estacion Migratoria,INM,Estacion Migratoria INM,28.642257,-106.084403,PG371,Chihuahua,Chihuahua
8,Ciudad Cuauhtemoc Estacion Provisional,INM,Estacion Migratoria INM,15.667668,-92.002344,PG736,Ciudad Cuauhtemoc,Chiapas
9,Ciudad del Carmen Estacion Provisional,INM,Estacion Migratoria INM,18.642626,-91.823607,PG766,Ciudad del Carmen,Campeche


In [69]:
# Usamos str.extract() con una expresión regular para manejar ambos casos: "Estacion" y "Estancia"
# La expresión regular '^(.*?)\s?(Estacion|Estancia)' captura cualquier texto desde el inicio de la cadena
# hasta antes de "Estacion" o "Estancia", ignorando si hay espacios antes de estas palabras
df1['ciudad'] = df1['name'].str.extract(r'^(.*?)\s?(Estacion|Estancia)', expand=False)[0]

# Muestra las primeras filas del DataFrame para verificar el resultado
print(df1[['name', 'ciudad']].head())


                                                name          ciudad
0                       Acayucan Estacion Migratoria        Acayucan
1                   Agua Prieta Estacion Provisional     Agua Prieta
2                 Aguascalientes Estacion Migratoria  Aguascalientes
3                      Campeche Estacion Provisional        Campeche
4  Centro de Atencion Integral al Trafico Fronter...             NaN


In [75]:
# Asignando manualmente los valores a la fila con índice 4
df1.at[4, 'ciudad'] = 'La Trinitaria'
#df1.at[4, 'estado'] = 'Chiapas'

# Asignando manualmente los valores a la fila con índice 5
df1.at[5, 'ciudad'] = 'Salto de Agua'
#df1.at[5, 'estado'] = 'Chiapas'

df1.at[14, 'estado'] = 'Tabasco'

In [44]:
ruta_archivo12 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Cargar el archivo .xls en un DataFrame
df_ciudades = pd.read_csv(ruta_archivo12)

In [47]:
df_ciudades

,ciudad,estado,ruta,detenciones,latitud,longitud,tipo,ID_Poligono,geometry,codigo_ruta,contador_xy,codigo_unico
0,"Ensenada, Baja California",Baja California,Ruta del Pacifico,29,31.865889,-116.602983,Presencia Migración Irregular,PG104,"POLYGON ((-117 32, -116 32, -116 31, -117 31, ...",P,1,P-PG104-1
1,"Playas de Rosarito, Baja California",Baja California,Ruta del Pacifico,38,32.258129,-116.873307,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,1,P-PG105-1
2,"Tecate, Baja California",Baja California,Ruta del Pacifico,594,32.565383,-116.629879,Presencia Migración Irregular,PG105,"POLYGON ((-117 33, -116 33, -116 32, -117 32, ...",P,2,P-PG105-2
3,"Mexicali, Baja California",Baja California,Ruta del Pacifico,0,32.640525,-115.474899,Presencia Migración Irregular,PG132,"POLYGON ((-116 33, -115 33, -115 32, -116 32, ...",P,1,P-PG132-1
4,"San Felipe, Baja California",Baja California,Ruta del Pacifico,4,31.026953,-114.839607,Presencia Migración Irregular,PG158,"POLYGON ((-115 32, -114 32, -114 31, -115 31, ...",P,1,P-PG158-1
...,...,...,...,...,...,...,...,...,...,...,...,...
384,"Tulum, Quintana Roo",Quintana Roo,Ruta Sur,13,20.169297,-87.449548,Presencia Migración Irregular,PG876,"POLYGON ((-88 21, -87 21, -87 20, -88 20, -88 ...",S,1,S-PG876-1
385,"Solidaridad, Quintana Roo",Quintana Roo,Ruta Sur,178,20.584375,-87.503453,Presencia Migración Irregular,PG876,"POLYGON ((-88 21, -87 21, -87 20, -88 20, -88 ...",S,2,S-PG876-2
386,"Cozumel, Quintana Roo",Quintana Roo,Ruta Sur,24,20.432060,-86.920691,Presencia Migración Irregular,PG903,"POLYGON ((-87 21, -86 21, -86 20, -87 20, -87 ...",S,1,S-PG903-1
387,"Benito Juarez, Quintana Roo",Quintana Roo,Ruta Sur,0,21.076681,-86.926335,Presencia Migración Irregular,PG904,"POLYGON ((-87 22, -86 22, -86 21, -87 21, -87 ...",S,1,S-PG904-1


In [72]:
# Asumiendo que df1 y df_ciudades ya están cargados

# Primero, nos aseguramos de que df_ciudades tenga valores únicos de 'ID_Poligono'
df_ciudades_unicos = df_ciudades.drop_duplicates(subset='ID_Poligono', keep='first')

# Ahora realizamos el merge, esta vez debería evitar la duplicación de filas en df1
df_merged = df1.merge(df_ciudades_unicos[['ID_Poligono', 'estado']], on='ID_Poligono', how='left')

# Renombramos las columnas si es necesario y eliminamos las adicionales que no necesitamos
# Este paso depende de si 'merge' crea nuevas columnas o no
df_merged.rename(columns={'estado_y': 'estado'}, inplace=True)
if 'estado_x' in df_merged.columns:
    df_merged.drop('estado_x', axis=1, inplace=True)

# Actualizamos df1 con los datos fusionados
df1 = df_merged

# Verificamos los resultados
print(df1[['ID_Poligono', 'estado']].head())



  ID_Poligono          estado
0       PG685        Veracruz
1       PG293          Sonora
2       PG472  Aguascalientes
3       PG794        Campeche
4       PG737         Chiapas


In [77]:
ruta_nueva5 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/estaciones INM/inm_gdp.csv'

# Guardar el DataFrame en un archivo Excel en la ruta especificada
df1.to_csv(ruta_nueva5, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_nueva5}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/estaciones INM/inm_gdp.csv
